In [ ]:
#!pip install git+https://github.com/brianmanderson/PyESAPI

In [ ]:
import pyesapi
import atexit
import SimpleITK as sitk
import os
import numpy as np

In [ ]:
def get_direction(varian_image):
    x_direction = [varian_image.get_XDirection()[i] for i in range(3)]
    y_direction = [varian_image.get_YDirection()[i] for i in range(3)]
    z_direction = [varian_image.get_ZDirection()[i] for i in range(3)]
    direction = x_direction + y_direction + z_direction
    return tuple(direction)

def get_spacing(varian_image):
    return (varian_image.get_XRes(), varian_image.get_YRes(), varian_image.get_ZRes())

def get_origin(potential_image):
    return tuple([potential_image.get_Origin()[i] for i in range(3)])

In [ ]:
def create_sitk_Image(varian_image):
    intercept = varian_image.VoxelToDisplayValue(0)
    image_array = varian_image.np_array_like().transpose()
    image_handle = sitk.GetImageFromArray(image_array + intercept)
    image_handle.SetSpacing(get_spacing(varian_image))
    image_handle.SetDirection(get_direction(varian_image))
    image_handle.SetOrigin(get_origin(varian_image))
    return image_handle

In [ ]:
def create_sitk_Mask(varian_mask, varian_image):
    image_handle = sitk.GetImageFromArray(varian_mask.np_mask_like(varian_image).transpose())
    image_handle.SetSpacing(get_spacing(varian_image))
    image_handle.SetDirection(get_direction(varian_image))
    image_handle.SetOrigin(get_origin(varian_image))
    return image_handle

In [ ]:
app = pyesapi.CustomScriptExecutable.CreateApplication('python_demo')
atexit.register(app.Dispose)

In [ ]:
#for pat_sum in app.PatientSummaries:
#    print(pat_sum.Id, pat_sum.LastName, pat_sum.FirstName)

In [ ]:
fid = open(os.path.join('.', 'PatientID'))
line = fid.readline()
line = fid.readline() # Go to the second patient
fid.close()
MRN = line.strip('\n')

In [ ]:
patient = app.OpenPatientById(MRN)

In [ ]:
courses = [c.Id for c in patient.Courses]

In [ ]:
plan = patient.CoursesLot(courses[1]).PlanSetupsLot(0)

In [ ]:
patient.CoursesLot(courses[1]).PlanSetupsLot(1).Id

In [ ]:
type(plan.Dose)

In [ ]:
print(f'Plan Id: {plan.Id}')
print(f'Dose Per Fx: {plan.PrescribedDosePerFraction}')
print(f'Number of Fx: {plan.NumberOfFractions}')

In [ ]:
np.min(beams[0].Dose.np_array_like() == plan.Dose.np_array_like())

In [ ]:
beams[0].Dose.get_ZRes()

In [ ]:
print(f'StructureID,TYPE,VOLUME')
for structure in plan.StructureSet.Structures:
    print(f'{structure.Id},{structure.DicomType},{structure.Volume:.2f}')

In [ ]:
image_handle = create_sitk_Image(plan.StructureSet.Image)

In [ ]:
plan.StructureSet.Image.VoxelToDisplayValue(0)

In [ ]:
sitk.WriteImage(image_handle, r'K:\Image.nii.gz')

In [ ]:
mask_handle = create_sitk_Mask(plan.StructureSet.StructuresLot('Liver'), plan.StructureSet.Image)

In [ ]:
dir(plan.StructureSet.Image)

In [ ]:
plan.StructureSet.Image.VoxelToDisplayValue()

In [ ]:
sitk.WriteImage(mask_handle, r'K:\Mask.nii.gz')

In [ ]:
dose_handle = create_sitk_Image(plan.Dose)

In [ ]:
sitk.WriteImage(dose_handle, r'K:\Dose.nii.gz')

In [ ]:
for i in range(len(beams)):
    if beams[i].Dose is None:
        continue
    beam_handle = create_sitk_Image(beams[i].Dose)
    sitk.WriteImage(dose_handle, r'K:\{}.nii.gz'.format(beams[i].Id))

In [ ]:
plan.StructureSet.StructuresLot('Liver').Volume # cc